In [2]:
import paddle.fluid as fluid
import paddle
from paddle.dataset import uci_housing
import numpy as np

In [4]:
paddle.enable_static()
x = fluid.layers.data(name='x', shape=[13], dtype='float32')
y = fluid.layers.data(name='y', shape=[1], dtype='float32')
hidden = fluid.layers.fc(input=x, size=100, act='relu')
y_predict = fluid.layers.fc(input=hidden, size=1, act=None)

cost = fluid.layers.square_error_cost(input=y_predict, label=y)
avg_cost = fluid.layers.mean(cost)

optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.001)
opts = optimizer.minimize(avg_cost)

place = fluid.CPUPlace()
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())

train_reader = fluid.io.batch(reader=uci_housing.train(),batch_size=128)

feeder = fluid.DataFeeder(place=place, feed_list=[x, y])

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: 
API "paddle.dataset.uci_housing.train" is deprecated since 2.0.0, and will be removed in future versions. Please use "paddle.text.datasets.UCIHousing" instead.
reason: Please use new dataset API which supports paddle.io.DataLoader 
Cache file /home/aistudio/.cache/paddle/dataset/uci_housing/housing.data not found, downloading http://paddlemodels.bj.bcebos.com/uci_housing/housing.data 
Begin to download
............
Download finished


In [5]:
for pass_id in range(1000):
    train_cost = 0
    for data in train_reader():
        train_cost = exe.run(program = fluid.default_main_program(),
                            feed = feeder.feed(data),
                            fetch_list = [avg_cost])
    if pass_id % 100 == 0:
        print("Pass:%d, Cost:%0.5f" % (pass_id, train_cost[0][0],))

params_dirname = "result"
fluid.io.save_inference_model(params_dirname, ['x'], [y_predict], exe)

Pass:0, Cost:119.96336
Pass:100, Cost:12.19085
Pass:200, Cost:13.41303
Pass:300, Cost:13.44936
Pass:400, Cost:13.08464
Pass:500, Cost:12.80711
Pass:600, Cost:12.58008
Pass:700, Cost:12.09806
Pass:800, Cost:11.77380
Pass:900, Cost:11.38116


['save_infer_model/scale_0.tmp_0']

In [7]:

infer_exe = fluid.Executor(place)
inference_scope = fluid.core.Scope()

with fluid.scope_guard(inference_scope):
    [inference_program, feed_target_names, fetch_targets
        ] = fluid.io.load_inference_model(params_dirname, infer_exe)
    batch_size = 10

    infer_reader = paddle.batch(
            paddle.dataset.uci_housing.test(), batch_size=batch_size)

    infer_data = next(infer_reader())
    infer_feat = np.array(
            [data[0] for data in infer_data]).astype("float32")
    
    infer_label = np.array(
            [data[1] for data in infer_data]).astype("float32")

    
    assert feed_target_names[0] == 'x'
    
    results = infer_exe.run(
            inference_program,
            feed={feed_target_names[0]: np.array(infer_feat)},
            fetch_list=fetch_targets)

    
    print("infer results: (House Price)")
    
    for idx, val in enumerate(results[0]):
        print("%d: %.2f" % (idx, val))

    print("\nground truth:")
    
    for idx, val in enumerate(infer_label):
        print("%d: %.2f" % (idx, val))

infer results: (House Price)
0: 7.53
1: 7.11
2: 8.88
3: 18.86
4: 10.58
5: 17.97
6: 19.26
7: 16.54
8: 7.76
9: 10.99

ground truth:
0: 8.50
1: 5.00
2: 11.90
3: 27.90
4: 17.20
5: 27.50
6: 15.00
7: 17.20
8: 17.90
9: 16.30


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
API "paddle.dataset.uci_housing.test" is deprecated since 2.0.0, and will be removed in future versions. Please use "paddle.text.datasets.UCIHousing" instead.
reason: Please use new dataset API which supports paddle.io.DataLoader 
  # Remove the CWD from sys.path while we load stuff.
